In [2]:
import sys
import os
from IPython.display import display, HTML, Markdown

from msticpy.nbtools.nbinit import init_notebook
init_notebook(namespace=globals());

In [3]:
qry_prov = QueryProvider("AzureSentinel")
qry_prov.connect(WorkspaceConfig())

Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [209]:
winsec_df = qry_prov.exec_query("SecurityEvent | sample 10000")
off_df = qry_prov.exec_query("OfficeActivity | sample 10000")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
commsec_df = qry_prov.exec_query("CommonSecurityLog | sample 10000")

<IPython.core.display.Javascript object>

In [5]:
data_df.columns

Index(['TenantId', 'TimeGenerated', 'SourceSystem', 'Account', 'AccountType',
       'Computer', 'EventSourceName', 'Channel', 'Task', 'Level',
       ...
       'VendorIds', 'Workstation', 'WorkstationName', 'PartitionKey', 'RowKey',
       'StorageAccount', 'AzureDeploymentID', 'AzureTableName', 'Type',
       '_ResourceId'],
      dtype='object', length=225)

In [7]:
# off_df[off_df["OfficeWorkload"].str.startswith('Shar')]

# Analyze data and create groupings based on blank/non-blank columns

In [67]:

input_df = commsec_df

# get col list
input_cols = list(input_df.columns)
# change any NAs to blank strings (this might cause TypeErrors !)
input_df = input_df.fillna("")
# Create a DF with True if a col value is blank
blank_cols = input_df == ""

# save the index as "orig_index" column
blank_cols = blank_cols.reset_index().rename(columns={"index": "orig_index"})

# remove one col - since we need something to count() in our group_by
# statement
# TenantId is a good choice since we know it's always non-blank
# and the same value
input_cols.remove("TenantId")
# group the bool DF by all the cols
record_grps = (
    blank_cols
    .groupby(list(input_cols))
    .count()
    .reset_index()  # reset index to get back a DF
    .reset_index()  # reset again to create an "index" column
    .rename(columns={"TenantId": "count", "index": "group_index"})
    .drop(columns=["orig_index"])
)
display("These are our groupings based on blank/non-blank columns")
display(record_grps)

# merge bool DF with the grouped results
blank_merge = blank_cols.merge(record_grps, on=input_cols, how="inner")
assert len(blank_merge) == len(blank_cols)

# create an orig_index column from the original index
# Note it's prob faster to leave this and merge using left_index=True, right_on="orig_index"
orig_data = input_df.reset_index().rename(columns={"index": "orig_index"})
# Now merge the result with the original data
# each record will now have a "group_index" column that maps to 
# its grouping in the record_grps data
input_df_blank_merge = orig_data.merge(blank_merge[["orig_index", "group_index", "count"]], on="orig_index")

'These are our groupings based on blank/non-blank columns'

,group_index,SourceSystem,TimeGenerated,ReceiptTime,DeviceVendor,DeviceProduct,DeviceEventClassID,LogSeverity,OriginalLogSeverity,DeviceAction,SimplifiedDeviceAction,Computer,CommunicationDirection,DeviceFacility,DestinationPort,DestinationIP,DeviceAddress,DeviceName,Message,Protocol,SourcePort,SourceIP,RemoteIP,RemotePort,MaliciousIP,...,DeviceCustomString4Label,DeviceCustomString5,DeviceCustomString5Label,DeviceCustomString6,DeviceCustomString6Label,DeviceCustomDate1,DeviceCustomDate1Label,DeviceCustomDate2,DeviceCustomDate2Label,FlexDate1,FlexDate1Label,FlexNumber1,FlexNumber1Label,FlexNumber2,FlexNumber2Label,FlexString1,FlexString1Label,FlexString2,FlexString2Label,AdditionalExtensions,StartTime,EndTime,Type,_ResourceId,count
0,0,False,False,False,False,False,False,False,True,True,True,True,True,True,True,False,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,3383
1,1,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,10
2,2,False,False,True,False,False,False,False,True,False,False,True,False,True,True,False,True,True,True,True,True,False,True,True,False,...,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,False,1
3,3,False,False,True,False,False,False,False,True,False,False,True,False,True,True,False,True,True,True,True,True,False,True,True,False,...,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,False,1
4,4,False,False,True,False,False,False,False,True,False,False,True,False,True,True,False,True,True,True,True,True,False,True,True,True,...,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,False,1426
5,5,False,False,True,False,False,False,False,True,False,False,True,False,True,True,False,True,True,True,True,True,False,True,True,True,...,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,False,153
6,6,False,False,True,False,False,False,False,True,False,False,True,False,True,True,False,True,True,True,True,True,False,True,True,True,...,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,False,1060
7,7,False,False,True,False,False,False,False,True,False,False,True,False,True,True,False,True,True,True,True,True,False,True,True,True,...,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,False,112
8,8,False,False,True,False,False,False,False,True,True,True,True,False,True,False,False,True,True,False,False,False,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,False,813
9,9,False,False,True,False,False,False,False,True,True,True,True,False,True,False,False,True,True,False,False,False,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,False,1112


# Clustering approach using DB Scan

In [221]:
winsec_df.head()

,TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,AccessList,AccessMask,AccessReason,AccountDomain,AccountExpires,AccountName,AccountSessionIdentifier,...,TargetUserName,TargetUserSid,TemplateContent,TemplateDSObjectFQDN,TemplateInternalName,TemplateOID,TemplateSchemaVersion,TemplateVersion,TokenElevationType,TransmittedServices,UserAccountControl,UserParameters,UserPrincipalName,UserWorkstations,VirtualAccount,VendorIds,Workstation,WorkstationName,PartitionKey,RowKey,StorageAccount,AzureDeploymentID,AzureTableName,Type,_ResourceId
0,8ecf8077-cf51-4820-aadd-14040956f35d,2021-07-30 20:11:34.007000+00:00,OpsManager,SECCXP\ADFS01$,Machine,ADFS01.seccxp.ninja,Microsoft-Windows-Security-Auditing,Security,13312,8,,4688,4688 - A new process has been created.,20ca974d-fade-4e11-abc4-6fc11997180a,1b4316df-f13f-4d52-9f01-ec133c28dc3d,00000000-0000-0000-0000-000000000001,2021-07-30 20:11:46.528000+00:00,AOI-8ecf8077-cf51-4820-aadd-14040956f35d,,,,,,,,...,-,S-1-0-0,,,,,,,%%1936,,,,,,,,,,,,,,,SecurityEvent,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/simuland/providers/microsoft....
1,8ecf8077-cf51-4820-aadd-14040956f35d,2021-07-30 20:11:34.023000+00:00,OpsManager,SECCXP\ADFS01$,Machine,ADFS01.seccxp.ninja,Microsoft-Windows-Security-Auditing,Security,13312,8,,4688,4688 - A new process has been created.,20ca974d-fade-4e11-abc4-6fc11997180a,7026d51a-6f55-41cc-bec1-33f0610411f2,00000000-0000-0000-0000-000000000001,2021-07-30 20:11:46.528000+00:00,AOI-8ecf8077-cf51-4820-aadd-14040956f35d,,,,,,,,...,-,S-1-0-0,,,,,,,%%1936,,,,,,,,,,,,,,,SecurityEvent,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/simuland/providers/microsoft....
2,8ecf8077-cf51-4820-aadd-14040956f35d,2021-07-30 20:11:34.047000+00:00,OpsManager,SECCXP\ADFS01$,Machine,ADFS01.seccxp.ninja,Microsoft-Windows-Security-Auditing,Security,12801,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Subje...",4663,4663 - An attempt was made to access an object.,20ca974d-fade-4e11-abc4-6fc11997180a,9191214d-c0de-47b4-8fc0-151f9a76a1fb,00000000-0000-0000-0000-000000000001,2021-07-30 20:11:46.528000+00:00,AOI-8ecf8077-cf51-4820-aadd-14040956f35d,%%4432 \t\t\t\t,0x1,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,SecurityEvent,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/simuland/providers/microsoft....
3,8ecf8077-cf51-4820-aadd-14040956f35d,2021-06-08 12:53:50.820000+00:00,OpsManager,SHIR-Hive\administrator,User,SHIR-Hive,Microsoft-Windows-Security-Auditing,Security,12544,16,,4625,4625 - An account failed to log on.,df6a58ab-e4d5-47e3-853f-26e261ec7efb,381724b3-851f-4ad4-8ff4-f19bf0321473,00000000-0000-0000-0000-000000000001,2021-06-08 12:53:52.646000+00:00,AOI-8ecf8077-cf51-4820-aadd-14040956f35d,,,,,,,,...,administrator,S-1-0-0,,,,,,,,-,,,,,,,,-,,,,,,SecurityEvent,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/soc-purview/providers/microso...
4,8ecf8077-cf51-4820-aadd-14040956f35d,2021-07-30 20:11:34.193000+00:00,OpsManager,SECCXP\ADFS01$,Machine,ADFS01.seccxp.ninja,Microsoft-Windows-Security-Auditing,Security,12801,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Subje...",4663,4663 - An attempt was made to access an object.,20ca974d-fade-4e11-abc4-6fc11997180a,b70a830a-df0f-44aa-9cb3-c03d067c3de2,00000000-0000-0000-0000-000000000001,2021-07-30 20:11:46.528000+00:00,AOI-8ecf8077-cf51-4820-aadd-14040956f35d,%%4432 \t\t\t\t,0x1,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,SecurityEvent,/subscriptions/d1d8779d-38d7-4f06-91db-9cbc8de0176f/resourcegroups/simuland/providers/microsoft....


In [210]:
input_df = winsec_df

# get col list
input_cols = list(input_df.columns)
# change any NAs to blank strings (this might cause TypeErrors !)
input_df = input_df.fillna("")
# Create a DF with True if a col value is blank
blank_cols = input_df == ""



## Cluster params (strength = EPS)

In [108]:
w_layout = widgets.Layout(width="50%")
w_style = {"description_width": "200px"}
cluster_strength = widgets.FloatSlider(
    description="Cluster strength",
    min=0.1,
    max=5.0,
    value=2.0,
    step=0.1,
    layout=w_layout,
    style=w_style,
)

min_cluster_size = widgets.IntSlider(
    description="Min cluster size",
    min=1,
    max=30,
    step=1,
    value=10,
    layout=w_layout,
    style=w_style,
)
display(cluster_strength)
display(min_cluster_size)

FloatSlider(value=2.0, description='Cluster strength', layout=Layout(width='50%'), max=5.0, min=0.1, style=Sli…

IntSlider(value=10, description='Min cluster size', layout=Layout(width='50%'), max=30, min=1, style=SliderSty…

In [211]:
blank_cols.head()

import numpy as np
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

X = blank_cols
# X = StandardScaler().fit_transform(X)

# #############################################################################
# Compute DBSCAN
print("Fitting cluster...")
db = DBSCAN(eps=cluster_strength.value, min_samples=min_cluster_size.value).fit(X)

labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

unique_labels = set(labels)


Fitting cluster...
Estimated number of clusters: 19
Estimated number of noise points: 40


In [232]:
def create_cluster_summary(input_df, cluster, labels):
    col_dict = {}
    cluster_rows = input_df[labels == cluster]
    for col in input_df.columns:
        try:
            n_unique = cluster_rows[col].nunique()
            dist_vals = cluster_rows[col].unique()
        except TypeError:
            n_unique = 999999
            dist_vals = ["Object types"]
        
        if n_unique <= 10:
            if n_unique == 1 and dist_vals[0] == "":
                continue
            values = ", ".join(sorted(str(val) for val in dist_vals))
        else:
            values = ", ".join(sorted(str(val) for val in dist_vals[:5]))
        col_dict["total_rows"] = len(cluster_rows)
        col_dict[f"{col}_count"] = n_unique
        col_dict[f"{col}_sample"] = values
        
    return col_dict                        

df_dict = {}
print("Summarizing clusters...")
for label in unique_labels:
    print(label, end=".")
    df_dict[label] = create_cluster_summary(input_df, label, labels)
    
summary_df = pd.DataFrame(df_dict).T
print("\ndone")


Summarizing clusters...
0.1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.-1.
done


In [235]:
summary_df.head()

,total_rows,TenantId_count,TenantId_sample,TimeGenerated_count,TimeGenerated_sample,SourceSystem_count,SourceSystem_sample,Account_count,Account_sample,AccountType_count,AccountType_sample,Computer_count,Computer_sample,EventSourceName_count,EventSourceName_sample,Channel_count,Channel_sample,Task_count,Task_sample,Level_count,Level_sample,EventID_count,EventID_sample,Activity_count,Activity_sample,...,PackageName_sample,Workstation_count,Workstation_sample,CallerProcessId_count,CallerProcessId_sample,CallerProcessName_count,CallerProcessName_sample,TargetSid_count,TargetSid_sample,AccessReason_count,AccessReason_sample,RelativeTargetName_count,RelativeTargetName_sample,ServiceName_count,ServiceName_sample,ShareLocalPath_count,ShareLocalPath_sample,ShareName_count,ShareName_sample,TargetInfo_count,TargetInfo_sample,TargetLogonGuid_count,TargetLogonGuid_sample,TargetServerName_count,TargetServerName_sample
0,973,1,8ecf8077-cf51-4820-aadd-14040956f35d,958,"2021-06-20 19:43:32.007000+00:00, 2021-07-30 20:11:34.007000+00:00, 2021-07-30 20:11:34.023000+0...",1,OpsManager,22,"CONTOSO\ADMINPC$, CONTOSO\VICTIMPC$, SECCXP\ADFS01$, SECCXP\DC01$, WORKGROUP\VictimPC2$",2,"Machine, User",18,"ADFS01.seccxp.ninja, AdminPc.Contoso.Azure, DC01.seccxp.ninja, VictimPC2, VictimPc.Contoso.Azure",1,Microsoft-Windows-Security-Auditing,1,Security,1,13312,1,8,1,4688,1,4688 - A new process has been created.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,709,1,8ecf8077-cf51-4820-aadd-14040956f35d,511,"2021-07-24 04:56:20.880000+00:00, 2021-07-30 20:11:34.047000+00:00, 2021-07-30 20:11:34.193000+0...",1,OpsManager,17,"NT AUTHORITY\LOCAL SERVICE, SECCXP\ADFS01$, SECCXP\ContosoAdmin, SECCXP\DC01$, SECCXP\WORKSTATION6$",2,"Machine, User",12,"ADFS01.seccxp.ninja, AdminPc.Contoso.Azure, DC01.seccxp.ninja, WIN2019SVR, WORKSTATION6.seccxp.n...",1,Microsoft-Windows-Security-Auditing,1,Security,3,"12800, 12801, 12802",1,8,1,4663,1,4663 - An attempt was made to access an object.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1030,1,8ecf8077-cf51-4820-aadd-14040956f35d,1030,"2021-05-23 16:06:22.127000+00:00, 2021-06-08 12:53:50.820000+00:00, 2021-06-10 11:48:00.557000+0...",1,OpsManager,106,"SHIR-Hive\administrator, SOC-FW-RDP\Admin, SOC-FW-RDP\Administrator, TAMACC.TAMARINDO.LIVE\Admin...",1,User,3,"SHIR-Hive, SHIR-SAP, SOC-FW-RDP",1,Microsoft-Windows-Security-Auditing,1,Security,1,12544,1,16,1,4625,1,4625 - An account failed to log on.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3329,1,8ecf8077-cf51-4820-aadd-14040956f35d,3329,"2021-05-03 03:05:56.280000+00:00, 2021-06-07 15:35:09.337000+00:00, 2021-06-16 21:18:08.573000+0...",1,OpsManager,871,"\ADMINISTRATOR, \AZUREUSER, \ONLANADMIN, \gadmin, \server",1,User,4,"SHIR-Hive, SHIR-SAP, SOC-FW-RDP, win10",1,Microsoft-Windows-Security-Auditing,1,Security,1,12544,1,16,1,4625,1,4625 - An account failed to log on.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,183,1,8ecf8077-cf51-4820-aadd-14040956f35d,176,"2021-07-30 20:11:34.233000+00:00, 2021-07-30 20:11:38.830000+00:00, 2021-07-30 20:11:38.837000+0...",1,OpsManager,7,"NT AUTHORITY\LOCAL SERVICE, SECCXP\ADFS01$, SECCXP\ContosoAdmin, SECCXP\DC01$, SECCXP\KDickens, ...",2,"Machine, User",3,"ADFS01.seccxp.ninja, DC01.seccxp.ninja, WORKSTATION6.seccxp.ninja",1,Microsoft-Windows-Security-Auditing,1,Security,1,13313,1,8,1,4689,1,4689 - A process has exited.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [238]:
def get_cluster_descriptions(summary_df):
    # determine columns that have the same value in the whole dataframe
    static_cols =  []
    for col in summary_df.filter(regex=".*_sample").columns:
        if summary_df[col].value_counts().values[0] == len(summary_df):
            static_cols.append(col.replace("_sample", "_count"))

    count_summary = summary_df.filter(regex=".*_count|total_rows")
    desc_summary = {}
    for idx, row in count_summary.iterrows():
        desc_cols = []
        for col in count_summary.columns:
            if col in static_cols or col == "total_rows":
                # ignore static columns and our total column
                continue
            if len(desc_cols) > 5:
                # only include 5 items in our description
                break
            if row[col] == 1:
                # For unique values in this set
                sample_col = col.replace("_count", "_sample")
                col_value = summary_df.iloc[idx][sample_col]
                if isinstance(col_value, str) and not col_value.isnumeric():
                    desc_cols.append(col_value[:50])
        if not desc_cols:
            desc_cols = ["No unique values"]
        desc = f"{'-'.join(desc_cols)} [{row.total_rows} rows]"
        # handle duplicate keys
        if desc in desc_summary:
            for i in range(len(summary_df)):
                if f"{desc}#{i}" not in desc_summary:
                    desc = f"{desc}#{i}"
                    break
            
         
        desc_summary[desc] = idx
    return desc_summary
    
get_cluster_descriptions(summary_df)

{'Microsoft-Windows-Security-Auditing-Security-4688 - A new process has been created. [973 rows]': 0,
 'Microsoft-Windows-Security-Auditing-Security-4663 - An attempt was made to access an object.-Security [709 rows]': 1,
 'User-Microsoft-Windows-Security-Auditing-Security-4625 - An account failed to log on.---0x0 [1030 rows]': 2,
 'User-Microsoft-Windows-Security-Auditing-Security-4625 - An account failed to log on.---0x0 [3329 rows]': 3,
 'Microsoft-Windows-Security-Auditing-Security-4689 - A process has exited. [183 rows]': 4,
 'User-Microsoft-Windows-AppLocker-Microsoft-Windows-AppLocker/EXE and DLL-8002 - A process was allowed to run. [822 rows]': 5,
 'Microsoft-Windows-Security-Auditing-Security-4673 - A privileged service was called. [207 rows]': 6,
 'Machine-Microsoft-Windows-Security-Auditing-Security-4662 - An operation was performed on an object.-- [565 rows]': 7,
 'Security [818 rows]': 8,
 'Machine-Microsoft-Windows-Security-Auditing-Security-SECCXP-0x3e7-S-1-5-18 [53 rows

In [229]:
"1234"[:40]

'1234'

In [255]:
# select_cluster = widgets.Select(
#     description="Cluster",
#     options=[str(label) for label in unique_labels]
# )

def rename_col(col):
    return col.replace("_count", "") if col.endswith("_count") else col.replace("_sample", "")


count_col_name = lambda x: "CountDistinctValues"
sample_col_name = lambda x: "Sample"

def format_table(data):
    counts = (
        pd.DataFrame(data)
        .rename(columns=count_col_name)
        .T
        .filter(regex=".*_count")
        .rename(columns=rename_col)
    )
    sample = (
        pd.DataFrame(data)
        .rename(columns=sample_col_name)
        .T
        .filter(regex=".*_sample")
        .rename(columns=rename_col)
    )

    return pd.concat([counts, sample]).T.dropna(how="all")

summary_dict = get_cluster_descriptions(summary_df)

disp_obj = None

def display_cluster(change=None):
    global disp_obj
    del change
    idx = select_cluster.value
    if disp_obj:
        disp_obj.update(format_table(summary_df.loc[idx]))
    else:
        return display(format_table(summary_df.loc[idx]), display_id=True)
                        

select_cluster = widgets.Select(
    description="Select cluster to view",
    options=get_cluster_descriptions(summary_df),
    layout=widgets.Layout(height="200px", width="100%"),
)



display(select_cluster)
disp_obj = display_cluster()
select_cluster.observe(display_cluster, names="value")

Select(description='Select cluster to view', layout=Layout(height='200px', width='100%'), options={'Microsoft-…

,CountDistinctValues,Sample
TenantId,1,8ecf8077-cf51-4820-aadd-14040956f35d
TimeGenerated,3329,"2021-05-03 03:05:56.280000+00:00, 2021-06-07 15:35:09.337000+00:00, 2021-06-16 21:18:08.573000+0..."
SourceSystem,1,OpsManager
Account,871,"\ADMINISTRATOR, \AZUREUSER, \ONLANADMIN, \gadmin, \server"
AccountType,1,User
Computer,4,"SHIR-Hive, SHIR-SAP, SOC-FW-RDP, win10"
EventSourceName,1,Microsoft-Windows-Security-Auditing
Channel,1,Security
Task,1,12544
Level,1,16


### Saving/retrieving summary data

In [254]:
import json

# we probably want to save these in a single file, rather than one per table
def save_table_summary(table_name, data):
    summary_dict = {table_name: json.loads(data.to_json())}

    with open(f"{table_name}.json", "w") as j_file:
        j_file.write(json.dumps(summary_dict))


save_table_summary("windows-security", summary_df)

def read_table_summary(table_name):
    
    with open(f"{table_name}.json", "r") as j_file:
        summary_dict = json.loads(j_file.read())
        
    df_json = summary_dict[table_name]
    return pd.DataFrame(df_json)

sum_df2 = read_table_summary("windows-security")
sum_df2.head()

,total_rows,TenantId_count,TenantId_sample,TimeGenerated_count,TimeGenerated_sample,SourceSystem_count,SourceSystem_sample,Account_count,Account_sample,AccountType_count,AccountType_sample,Computer_count,Computer_sample,EventSourceName_count,EventSourceName_sample,Channel_count,Channel_sample,Task_count,Task_sample,Level_count,Level_sample,EventID_count,EventID_sample,Activity_count,Activity_sample,...,PackageName_sample,Workstation_count,Workstation_sample,CallerProcessId_count,CallerProcessId_sample,CallerProcessName_count,CallerProcessName_sample,TargetSid_count,TargetSid_sample,AccessReason_count,AccessReason_sample,RelativeTargetName_count,RelativeTargetName_sample,ServiceName_count,ServiceName_sample,ShareLocalPath_count,ShareLocalPath_sample,ShareName_count,ShareName_sample,TargetInfo_count,TargetInfo_sample,TargetLogonGuid_count,TargetLogonGuid_sample,TargetServerName_count,TargetServerName_sample
0,973,1,8ecf8077-cf51-4820-aadd-14040956f35d,958,"2021-06-20 19:43:32.007000+00:00, 2021-07-30 20:11:34.007000+00:00, 2021-07-30 20:11:34.023000+0...",1,OpsManager,22.0,"CONTOSO\ADMINPC$, CONTOSO\VICTIMPC$, SECCXP\ADFS01$, SECCXP\DC01$, WORKGROUP\VictimPC2$",2.0,"Machine, User",18,"ADFS01.seccxp.ninja, AdminPc.Contoso.Azure, DC01.seccxp.ninja, VictimPC2, VictimPc.Contoso.Azure",1,Microsoft-Windows-Security-Auditing,1,Security,1,13312,1,8,1,4688,1,4688 - A new process has been created.,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
1,709,1,8ecf8077-cf51-4820-aadd-14040956f35d,511,"2021-07-24 04:56:20.880000+00:00, 2021-07-30 20:11:34.047000+00:00, 2021-07-30 20:11:34.193000+0...",1,OpsManager,17.0,"NT AUTHORITY\LOCAL SERVICE, SECCXP\ADFS01$, SECCXP\ContosoAdmin, SECCXP\DC01$, SECCXP\WORKSTATION6$",2.0,"Machine, User",12,"ADFS01.seccxp.ninja, AdminPc.Contoso.Azure, DC01.seccxp.ninja, WIN2019SVR, WORKSTATION6.seccxp.n...",1,Microsoft-Windows-Security-Auditing,1,Security,3,"12800, 12801, 12802",1,8,1,4663,1,4663 - An attempt was made to access an object.,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
2,1030,1,8ecf8077-cf51-4820-aadd-14040956f35d,1030,"2021-05-23 16:06:22.127000+00:00, 2021-06-08 12:53:50.820000+00:00, 2021-06-10 11:48:00.557000+0...",1,OpsManager,106.0,"SHIR-Hive\administrator, SOC-FW-RDP\Admin, SOC-FW-RDP\Administrator, TAMACC.TAMARINDO.LIVE\Admin...",1.0,User,3,"SHIR-Hive, SHIR-SAP, SOC-FW-RDP",1,Microsoft-Windows-Security-Auditing,1,Security,1,12544,1,16,1,4625,1,4625 - An account failed to log on.,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
3,3329,1,8ecf8077-cf51-4820-aadd-14040956f35d,3329,"2021-05-03 03:05:56.280000+00:00, 2021-06-07 15:35:09.337000+00:00, 2021-06-16 21:18:08.573000+0...",1,OpsManager,871.0,"\ADMINISTRATOR, \AZUREUSER, \ONLANADMIN, \gadmin, \server",1.0,User,4,"SHIR-Hive, SHIR-SAP, SOC-FW-RDP, win10",1,Microsoft-Windows-Security-Auditing,1,Security,1,12544,1,16,1,4625,1,4625 - An account failed to log on.,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None
4,183,1,8ecf8077-cf51-4820-aadd-14040956f35d,176,"2021-07-30 20:11:34.233000+00:00, 2021-07-30 20:11:38.830000+00:00, 2021-07-30 20:11:38.837000+0...",1,OpsManager,7.0,"NT AUTHORITY\LOCAL SERVICE, SECCXP\ADFS01$, SECCXP\ContosoAdmin, SECCXP\DC01$, SECCXP\KDickens, ...",2.0,"Machine, User",3,"ADFS01.seccxp.ninja, DC01.seccxp.ninja, WORKSTATION6.seccxp.ninja",1,Microsoft-Windows-Security-Auditing,1,Security,1,13313,1,8,1,4689,1,4689 - A process has exited.,...,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None


# End of cluster stuff

---

In [140]:

#     ( ,
#     pd.DataFrame(summary_df.loc[int(select_cluster.value)]).T.filter(regex=".*_sample").rename(columns=rename_col),
# ],
# ignore_)

,counts,sample
TenantId,1,8ecf8077-cf51-4820-aadd-14040956f35d
SourceSystem,1,OpsManager
TimeGenerated,2257,"2021-07-30 02:01:48.672000+00:00, 2021-07-30 02:01:48.673000+00:00, 2021-07-30 02:01:48.695000+0..."
DeviceVendor,1,Palo Alto Networks
DeviceProduct,1,PAN-OS
DeviceEventClassID,2,"end, start"
LogSeverity,1,1
CommunicationDirection,NaN,NaN
DestinationPort,NaN,NaN
DestinationIP,1,10.6.1.5


# Find the unique columns for each group

## TODO:

- We should prob ignore groups with very small total population (or get more data)
  - or we should treat them differently
  - with 1 or small number of records it's impossible to tell the difference
    between grouping columns and data columns
- Would be nice to remove columns that are the same value for all rows
- Turn the code into functions
- Better output storage, maybe also store the variability (n unique values)
- Try it on a big customer with more data

In [68]:
# Now we have to try to detect which cols in each
# group have the same value
agg_oper = "nunique"

def create_agg(data, agg_oper):
    # create a dict of column name and agg operation
    # TODO - we reference the global input_cols here - should be a param
    uniq_agg_dict = {col: agg_oper for col in input_cols}
    bad_types = []
    for col, agg in uniq_agg_dict.items():
        col_agg_dict = {col: agg}
        # Need to trap for exceptions - if a column is a list, etc. it isn't
        # hashable so unique/nunique will throw an exception
        try:
            input_df_blank_merge.groupby("group_index").agg(col_agg_dict)
        except TypeError:
            bad_types.append(col)

    if bad_types:
        print("Not hashable (unique-able) cols:", bad_types)
    # recreate the agg dict without the problem columns
    uniq_agg_dict2 = {col: agg_oper for col in input_cols if col not in bad_types}
    # return the result
    return data.groupby("group_index").agg(uniq_agg_dict2)


In [69]:
from collections import defaultdict

# something to store results in
group_cols = defaultdict(dict)
group_cols_counts = defaultdict(dict)
# aggregate by group for nunique and unique
uniq_val_count_df = create_agg(input_df_blank_merge, "nunique")
uniq_val_samples_df = create_agg(input_df_blank_merge, "unique")

# Iterate through our groups
for row in uniq_val_count_df.itertuples():
    # get the unique-ish columns
    unique_cols = [
        col
        for col, count in row._asdict().items()
        if count == 1   # threshold of uniqueness
        and col in record_grps.columns # legit column
        and not record_grps.loc[row.Index][col] # Is it none-blank for this group?
    ]
    small_var_cols = [
        col
        for col, count in row._asdict().items()
        if 10 > count > 1   # threshold of uniqueness
        and col in record_grps.columns # legit column
        and not record_grps.loc[row.Index][col] # Is it none-blank for this group?
    ]
    # get the variable columns
    data_cols = [
        col
        for col, count in row._asdict().items()
        if count > 10
        and col in record_grps.columns
        and not record_grps.loc[row.Index][col] # Is it none-blank for this group?
    ]
    group_cols[row.Index]["Num_rows"] = row.TimeGenerated
    # for each column with unique (small number of) values
    for col in unique_cols:
        # get sample data for this group/column
        sample_data = list(uniq_val_samples_df.loc[row.Index][col])
        # add to our record
        group_cols[row.Index][col] = sample_data[0]
        group_cols_counts[row.Index][col] = 1
    for col in small_var_cols:
        # get sample data for this group/column
        sample_data = ", ".join(str(item) for item in uniq_val_samples_df.loc[row.Index][col])
        # add to our record
        group_cols[row.Index][col] = sample_data
        group_cols_counts[row.Index][col] = len(uniq_val_samples_df.loc[row.Index][col])
    # append the list of data (variable-content) columns
    group_cols[row.Index]["Data columns"] = ", ".join(data_cols)

    
tables_df = pd.DataFrame(group_cols).T

In [71]:
tables_counts_df = pd.DataFrame(group_cols_counts).T


In [72]:
tables_counts_df.head()

,SourceSystem,ReceiptTime,DeviceVendor,DeviceProduct,LogSeverity,DestinationIP,DeviceVersion,Activity,DeviceExternalID,Type,DeviceEventClassID,TimeGenerated,DeviceAction,SimplifiedDeviceAction,CommunicationDirection,SourceIP,MaliciousIP,ThreatSeverity,IndicatorThreatType,ThreatConfidence,ReportReferenceLink,MaliciousIPLongitude,MaliciousIPLatitude,MaliciousIPCountry,ApplicationProtocol,...,DeviceCustomString5Label,DeviceCustomString6,DeviceCustomString6Label,AdditionalExtensions,RequestContext,Protocol,DeviceInboundInterface,DeviceOutboundInterface,DestinationPort,Message,DeviceFacility,DeviceAddress,DeviceName,ExternalID,SourceHostName,FlexNumber1,FlexNumber1Label,FlexNumber2,FlexNumber2Label,DestinationTranslatedAddress,DestinationTranslatedPort,SourcePort,SourceTranslatedPort,DestinationUserName,SourceProcessName
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,1.0,1.0,1.0,NaN,1.0,5.0,NaN,1.0,2.0,NaN,2.0,2.0,1.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,...,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
tables_df.sort_values("Num_rows", ascending=False).head()

,Num_rows,SourceSystem,ReceiptTime,DeviceVendor,DeviceProduct,LogSeverity,DestinationIP,DeviceVersion,Activity,DeviceExternalID,Type,DeviceEventClassID,Data columns,TimeGenerated,DeviceAction,SimplifiedDeviceAction,CommunicationDirection,SourceIP,MaliciousIP,ThreatSeverity,IndicatorThreatType,ThreatConfidence,ReportReferenceLink,MaliciousIPLongitude,MaliciousIPLatitude,...,DeviceCustomString5Label,DeviceCustomString6,DeviceCustomString6Label,AdditionalExtensions,RequestContext,Protocol,DeviceInboundInterface,DeviceOutboundInterface,DestinationPort,Message,DeviceFacility,DeviceAddress,DeviceName,ExternalID,SourceHostName,FlexNumber1,FlexNumber1Label,FlexNumber2,FlexNumber2Label,DestinationTranslatedAddress,DestinationTranslatedPort,SourcePort,SourceTranslatedPort,DestinationUserName,SourceProcessName
0,2257,OpsManager,$cefformatted-receive_time,Palo Alto Networks,PAN-OS,1,10.6.1.5,8.0.0,TRAFFIC,007057000050480,CommonSecurityLog,"end, start","TimeGenerated, SourceIP",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1238,OpsManager,NaN,Zscaler,NSSWeblog,3,NaN,5.7,"Allowed, Reputation block outbound request: malicious URL, Not allowed to browse this category, ...",NaN,CommonSecurityLog,"Allowed, Blocked","TimeGenerated, DestinationIP, DestinationServiceName, DestinationHostName, FileType, ReceivedByt...",NaN,"Allowed, Blocked","Allowed, Blocked",1,"3.88.48.125, 40.83.138.250, 72.52.82.194, 100.26.44.132, 1295336, 20.75.81.37, 9210267, 8758559",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,threatname,None,md5hash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,1193,OpsManager,NaN,Fortinet,Fortigate,3,10.0.0.5,v6.4.5,"traffic:forward client-rst, traffic:forward server-rst, traffic:forward timeout",FGVM4VTM21000723,CommonSecurityLog,00013,"TimeGenerated, SourcePort, SourceIP, ExternalID, ReceivedBytes, SentBytes, AdditionalExtensions",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6,port1,port2,3389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,3389,NaN,NaN,NaN,NaN
9,1112,OpsManager,NaN,Fortinet,Fortigate,2,10.0.1.6,v6.4.5,utm:app-ctrl signature pass,FGVM4VTM21000723,CommonSecurityLog,28704,"TimeGenerated, SourcePort, SourceIP, ExternalID, AdditionalExtensions",NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6,port1,port2,3389,"Remote.Access: RDP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1001,OpsManager,NaN,Zscaler,NSSWeblog,3,NaN,5.7,"Allowed, Reputation block outbound request: phishing site, Not allowed to browse this category",NaN,CommonSecurityLog,"Allowed, Blocked","TimeGenerated, DestinationIP, DestinationServiceName, DestinationHostName, ReceivedBytes, SentBy...",NaN,"Allowed, Blocked","Allowed, Blocked",1,"3.88.48.125, 72.52.82.194, 40.83.138.250, 100.26.44.132, 20.75.81.37, 10180635, 40.86.83.207, 93...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,threatname,None,md5hash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
global_cols = []
for col in tables_df.columns:
    if tables_df[col].value_counts().values[0] == len(tables_df):
        global_cols.append(col)

global_cols



['SourceSystem', 'Type']

In [76]:
multi_valued_cols = (tables_counts_df > 1).sum()
global_multi_val_cols = set(multi_valued_cols[multi_valued_cols > 0].index)

In [81]:
from collections import defaultdict
import numpy as np

table_counts = {}
global_other_cols = set()

for idx, row in tables_df.sort_values("Num_rows", ascending=False).iterrows():
#     if row.Num_rows < 10:
#         continue
    unique_cols = []
    other_cols = []
    for col in tables_counts_df.columns:
        if tables_counts_df.loc[idx][col] == 1:
            unique_cols.append(col)
        elif tables_counts_df.loc[idx][col] != np.nan:
            other_cols.append(col)

    gl_unique_cols = (set(sorted(unique_cols)) - set(global_cols)) - global_multi_val_cols
    table_counts[idx] = {
        "rows": row.Num_rows,
        "unique_cols": list(gl_unique_cols),
        "unique_vals": [row[u_col] for u_col in gl_unique_cols],
        "other_cols": other_cols,
        
    }
    
from pprint import pprint


table_counts = pd.DataFrame(table_counts).T
table_counts

,rows,unique_cols,unique_vals,other_cols
0,2257,"[ReceiptTime, DeviceExternalID, DeviceProduct, DeviceVendor]","[$cefformatted-receive_time, 007057000050480, PAN-OS, Palo Alto Networks]","[DeviceEventClassID, TimeGenerated, DeviceAction, SimplifiedDeviceAction, CommunicationDirection..."
4,1238,"[DeviceCustomString4Label, DeviceCustomString6Label, DeviceProduct, DeviceVendor, DeviceCustomSt...","[malwarecat, md5hash, NSSWeblog, Zscaler, malwareclass, None, riskscore, threatname, dept, None,...","[ReceiptTime, DestinationIP, Activity, DeviceExternalID, DeviceEventClassID, TimeGenerated, Devi..."
14,1193,"[DeviceProduct, DeviceVendor, DestinationTranslatedAddress, DeviceExternalID, DeviceInboundInter...","[Fortigate, Fortinet, 10.0.1.6, FGVM4VTM21000723, port1, port2, 3389.0]","[ReceiptTime, Activity, TimeGenerated, DeviceAction, SimplifiedDeviceAction, CommunicationDirect..."
9,1112,"[DeviceProduct, DeviceVendor, DeviceExternalID, DeviceInboundInterface, DeviceOutboundInterface]","[Fortigate, Fortinet, FGVM4VTM21000723, port1, port2]","[ReceiptTime, TimeGenerated, DeviceAction, SimplifiedDeviceAction, SourceIP, MaliciousIP, Threat..."
6,1001,"[DeviceCustomString4Label, DeviceCustomString6Label, DeviceProduct, DeviceVendor, DeviceCustomSt...","[malwarecat, md5hash, NSSWeblog, Zscaler, malwareclass, None, riskscore, threatname, dept, None,...","[ReceiptTime, DestinationIP, Activity, DeviceExternalID, DeviceEventClassID, TimeGenerated, Devi..."
8,810,"[DeviceProduct, DeviceVendor, DeviceExternalID, DeviceInboundInterface, DeviceOutboundInterface]","[Fortigate, Fortinet, FGVM4VTM21000723, port2, port1]","[ReceiptTime, DestinationIP, TimeGenerated, DeviceAction, SimplifiedDeviceAction, CommunicationD..."
15,705,"[DeviceProduct, DeviceVendor, DeviceExternalID, DeviceInboundInterface, DeviceOutboundInterface]","[Fortigate, Fortinet, FGVM4VTM21000723, port2, port1]","[ReceiptTime, DestinationIP, Activity, DeviceEventClassID, TimeGenerated, DeviceAction, Simplifi..."
5,125,"[DeviceCustomString4Label, DeviceCustomString6Label, DeviceProduct, DeviceVendor, DeviceCustomSt...","[malwarecat, md5hash, NSSWeblog, Zscaler, malwareclass, None, riskscore, threatname, dept, None,...","[ReceiptTime, DestinationIP, Activity, DeviceExternalID, DeviceEventClassID, TimeGenerated, Devi..."
7,106,"[DeviceCustomString4Label, DeviceCustomString6Label, DeviceProduct, DeviceVendor, DeviceCustomSt...","[malwarecat, md5hash, NSSWeblog, Zscaler, malwareclass, None, riskscore, threatname, dept, None,...","[ReceiptTime, DestinationIP, Activity, DeviceExternalID, DeviceEventClassID, TimeGenerated, Devi..."
13,11,"[DeviceProduct, DeviceVendor, DestinationTranslatedAddress, DeviceExternalID, DeviceInboundInter...","[Fortigate, Fortinet, 10.0.1.5, FGVM4VTM21000723, port1, port2, 514.0]","[ReceiptTime, TimeGenerated, DeviceAction, SimplifiedDeviceAction, CommunicationDirection, Sourc..."


In [80]:
table_counts.groupby(["unique_cols", "unique_vals"]).sum()

TypeError: unhashable type: 'list'

In [65]:
input_df.ResultStatus.unique()

array(['True', '', 'Succeeded'], dtype=object)

In [114]:
[col for col, count in row._asdict().items() if count > 2]

['TimeGenerated',
 'Account',
 'Computer',
 'EventData',
 'SourceComputerId',
 'EventOriginId',
 'TimeCollected',
 'AccessList',
 'AccessMask',
 'HandleId',
 'ObjectName',
 'ProcessId',
 'SubjectAccount',
 'SubjectUserName',
 '_224']

In [88]:
def comp_row(idx):
    ser1 = orig_data.iloc[idx]
    ser2 = blank_merge.iloc[idx]
    ser1.name = "value"
    ser2.name = "blank"
    display(pd.concat([ser1, ser2], axis=1).query("blank == False"))
    
# comp_row(2)
ser1

TenantId                                                                            8ecf8077-cf51-4820-aadd-14040956f35d
TimeGenerated                                                                           2021-06-23 15:23:13.027000+00:00
SourceSystem                                                                                                  OpsManager
Account                                                                                                   \administrator
AccountType                                                                                                         User
                                                                    ...                                                 
StorageAccount                                                                                                          
AzureDeploymentID                                                                                                       
AzureTableName                  

In [26]:
disp_cols = list(input_df_blank_merge.columns)
disp_cols = [
'EventSourceName',
 'Channel',
 'Task',
 'Level',
 'EventData',
 'EventID',
 'Activity',
 'SubjectLogonId',
 'SubjectUserName',
 'SubjectUserSid',
 'TableId',
 'TargetAccount',
 'TargetDomainName',
 'TargetUserName',
 'TargetUserSid',
    'group_index',
]

In [85]:
input_df_blank_merge[disp_cols].query("group_index == 3")

,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,SubjectLogonId,SubjectUserName,SubjectUserSid,TableId,TargetAccount,TargetDomainName,TargetUserName,TargetUserSid,group_index
153,Microsoft-Windows-Security-Auditing,Security,12292,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Subje...",5059,5059 - Key migration operation.,0x3e7,SHIR-Hive$,S-1-5-18,,,,,,3
1006,Microsoft-Windows-Security-Auditing,Security,12292,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Subje...",5059,5059 - Key migration operation.,0x3e7,SOC-FW-RDP$,S-1-5-18,,,,,,3
1233,Microsoft-Windows-Security-Auditing,Security,12292,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Subje...",5059,5059 - Key migration operation.,0x3e7,SHIR-SAP$,S-1-5-18,,,,,,3
3158,Microsoft-Windows-Security-Auditing,Security,12804,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Subje...",4702,4702 - A scheduled task was updated.,0x3e4,SOC-FW-RDP$,S-1-5-20,,,,,,3
4783,Microsoft-Windows-Security-Auditing,Security,12804,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Subje...",4702,4702 - A scheduled task was updated.,0x3e4,SHIR-SAP$,S-1-5-20,,,,,,3
6567,Microsoft-Windows-Security-Auditing,Security,12804,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Subje...",4702,4702 - A scheduled task was updated.,0x3e7,ADMINPC$,S-1-5-18,,,,,,3
8135,Microsoft-Windows-Security-Auditing,Security,12804,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Subje...",4702,4702 - A scheduled task was updated.,0x3e4,VICTIMPC$,S-1-5-20,,,,,,3
9327,Microsoft-Windows-Security-Auditing,Security,12292,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Subje...",5059,5059 - Key migration operation.,0x3e7,VICTIMPC$,S-1-5-18,,,,,,3
